In [23]:
from lotc.trainer import FileBinaryClassifierTrainer

pos_data = './data/pos'
neg_data = './data/neg'
trainer = FileBinaryClassifierTrainer(
    model_name='logistic_regression_example',
    max_length=4096,
    num_epochs=20,
    train_loss_threshold=4
)

In [ ]:
trainer.train(pos_data, neg_data)

In [ ]:
trainer.save()

In [ ]:
from lotc.embedding import long_text_embedding
model = trainer.load()
model.predict(long_text_embedding('这是一个测试文本.', max_length=4096, chunk_size=256)[1])